# Using Variables

### Introduction

In this lesson, we'll see how we can incorporate coding practice such as using variables to prevent repeating ourselves.  We'll then see how to use output variables to have terraform return to us valuable information after updating a resource.

### Adding Input Variables

Let's take another look at the EC2 resource and the HTTP security group that we configured in previous lessons.

```bash
resource "aws_instance" "backend" {
  ami           = "ami-0c55b159cbfafe1f0"
  instance_type = "t2.micro"

  user_data = <<-EOF
              #!/bin/bash
              echo "Hello, World" > index.html
              nohup busybox httpd -f -p 80 &
              EOF
}

resource "aws_security_group" "app_security" {
  name = "app_security"
  ingress {
    from_port   = 80
    to_port     = 80
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }
}

```

As we can see there is a good amount of repetition with specifying our port number of `80`.  This is a good case for using a variable.  Let's see it.

```bash
variable "port_number" {
  description = "server number for HTTP requests"
  type        = number
  default     = 80
}
```

And then we can access the variable by the object type, followed by the name of the variable.  So updating our security group resource, it looks like the following:

```bash
resource "aws_security_group" "app_security" {
  name = "app_security"
  ingress {
    from_port   = var.port_number
    to_port     = var.port_number
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }
}
```

We can also use the variable to update our aws security code.

```bash
resource "aws_instance" "backend" {
  ami           = "ami-0c55b159cbfafe1f0"
  instance_type = "t2.micro"

  user_data = <<-EOF
              #!/bin/bash
              echo "Hello, World" > index.html
              nohup busybox httpd -f -p "${var.port_number}" &
              EOF
}
```

So above, because we are referencing the variable from within our string, we use interpolation when referencing our variable -- `${var.port_number}`.

The above variables are called *input variables*.  And when we specified the value, we really specified the *default* value.

```bash
variable "port_number" {
  description = "server number for HTTP requests"
  type        = number
  default     = 80
}
```

But we can override this default variable through the terraform command line like so. 

> `terraform apply -var "port_number=3000"`

### Output Variables 

Now that we learned about input variables, the other type of variables we can use are output variables.  Let's see an example.

```bash
output "public_ip" {
  value       = aws_instance.backend.public_ip
  description = "The public IP of the web server"
}
```

So as we can see, output variables are variables that are computed from our resources.  These are valuable because after running `terraform apply`, now we'll be outputted the public ip address and will not have to search around for it.

### Summary

In this lesson, we learned how to define both input variables and output variables in terraform.  For input variables, we define them with the variable name, type and default value like so:

```bash
variable "port_number" {
  description = "server number for HTTP requests"
  type        = number
  default     = 80
}
```

And then we reference the variable by with var followed by the variable name, so for the above it's `var.port_number`.  We can also override the default value from the command line with something like `terraform apply var "port_number=3000"`.

Then we moved onto output variables, which are variables that are computed when configuring our resources, and outputted from the apply command.

```bash
output "public_ip" {
  value       = aws_instance.backend.public_ip
  description = "The public IP of the web server"
}
```